In [ ]:
import uproot as up
from hist import Hist, Stack
import numpy as np
import os
import pandas as pd
import awkward as ak
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from pathlib import Path
from collections import namedtuple
sns.set_style('white')
sns.set_context('poster')

In [ ]:
outdir = Path("/g/g92/samuele/nEXO/sensitivity/work/SensitivityPaper2020_scripts/SiteSelectionStudy/results/")

In [ ]:
Scenario = namedtuple('scenario', ['name', 'dir', 'muons_per_file'])

case_list = [Scenario('LNGS',
                      Path("/p/lustre2/nexouser/data/site_selection/LNGS_Shielding/large_run/cosmogenics_processed/data/"),
                      50e3),
             Scenario('SNOLAB',
                      Path("/p/lustre2/nexouser/data/site_selection/SNOLAB_Shielding_25TeV_v3/cosmo_analysis_data/data/"),
                      5e3)]

In [ ]:
branches = ['Xe137Count','Cu64Count','Cu66Count']
events_filter = '(Xe137Count >0) | (Cu64Count >0) | (Cu66Count >0)'

In [ ]:
%%time
for case in case_list:
    print(f"Loading data for {case.name}...")
    
    # if case.name == "LNGS":
    #     file_list = list(x for x in case.dir.rglob('*.root') if x.is_file() and x.stat().st_size>1.1e9)
    # else:
    file_list = list(x for x in case.dir.rglob('*.root') if x.is_file())        
    print(f'\tFiles available: {len(file_list)}')
    
    upfile = up.open(file_list[0], object_cache=2000, array_cache='2000 MB')
    tree = upfile["cosmoTree"]
    print(f'\tEntries in the first file: {tree.num_entries}')
    
    events = ak.to_dataframe(up.concatenate(file_list, 
                                        branches, events_filter, allow_missing=True,
                                        library='ak'))    
    print(events.head(),'\n')
    
    events.to_pickle(outdir / f"{case.name}_activation_products.pkl")

In [ ]:
# events = tree.arrays(branches, events_filter, library='pd')
# for array in up.iterate(file_list[1:], branches, events_filter, library='pd'):
    # events.append(array)

In [ ]:
# for file in file_list[1:]:
    # with up.open(folder_path / file_name, object_cache=2000, array_cache='2000 MB') as upfile:
        # print(file)
        # tree = upfile["cosmoTree"]
        # events.append(tree.arrays(branches, events_filter, library='pd'))

In [ ]:
events_dict = {}
for case in case_list:
    events_dict[case.name] = pd.read_pickle(outdir / f"{case.name}_activation_products.pkl")

In [ ]:
for case in case_list:
    events = events_dict[case.name]
    n_muons = case.muons_per_file * len(file_list)
    Xe136ncapture_prob = events['Xe137Count'][events.Xe137Count > 0].count()/n_muons
    print(f"Probability of >1 136Xe n-capture per muon at {case.name}: {Xe136ncapture_prob:.2e}")

In [ ]:
for name, events in events_dict.items():
    fig, ax = plt.subplots(figsize=(8, 6))
    p = sns.histplot(events[(events.Xe137Count<30) & (events.Cu66Count<30) & (events.Cu64Count<30)], 
                 x='Xe137Count', y='Cu64Count', cmap="cividis",  
                 cbar = True, binwidth=1, binrange=((0,30),(0,30)),
                 norm=colors.LogNorm(), vmin=None, vmax=None)
    p.set(xlabel='$^{136}$Xe n-captures/muon')
    p.set(ylabel='$^{63}$Cu n-captures/muon')
    p.set(title=name)
    fig.savefig(outdir / f"{name}_63Cu_vs_136Xe.png", bbox_inches="tight", transparent=False, facecolor='white')

In [ ]:
for name, events in events_dict.items():
    fig, ax = plt.subplots(figsize=(8, 6))
    p = sns.histplot(events[(events.Xe137Count<30) & (events.Cu66Count<30) & (events.Cu64Count<30)], 
                 x='Xe137Count', y='Cu66Count', cmap="cividis",  
                 cbar = True, binwidth=1, binrange=((0,30),(0,30)),
                 norm=colors.LogNorm(), vmin=None, vmax=None)
    p.set(xlabel='$^{136}$Xe n-captures/muon')
    p.set(ylabel='$^{65}$Cu n-captures/muon')
    p.set(title=name)
    fig.savefig(outdir / f"{name}_65Cu_vs_136Xe.png", bbox_inches="tight", transparent=False, facecolor='white')

In [ ]:
for name, events in events_dict.items():
    fig, ax = plt.subplots(figsize=(8, 6))
    p = sns.histplot(events[(events.Xe137Count<30) & (events.Cu66Count<30) & (events.Cu64Count<30)], 
                 x='Cu66Count', y='Cu64Count', cmap="cividis", 
                 cbar = True, binwidth=1, binrange=((0,30),(0,30)),
                 norm=colors.LogNorm(), vmin=None, vmax=None)
    p.set(xlabel='$^{65}$Cu n-captures/muon')
    p.set(ylabel='$^{63}$Cu n-captures/muon')
    p.set(title=name)
    fig.savefig(outdir / f"{name}_63Cu_vs_65Cu.png", bbox_inches="tight", transparent=False, facecolor='white')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
hist_dict = {} 
for name, events in events_dict.items():
    hist_dict[name] = Hist.new.Regular(150, 0, 150).Double()
    hist_dict[name].fill(events[events.Xe137Count>0]['Xe137Count'])
    hist_dict[name][:] = hist_dict[name].density()
    print(name)
    print(f"\t Mean: {events[events.Xe137Count>0]['Xe137Count'].mean()}")
    print(f"\t Median: {events[events.Xe137Count>0]['Xe137Count'].median()}")
s = Stack.from_dict(hist_dict)
s.plot(histtype="fill", alpha=0.7)
plt.legend()
plt.xlabel(r'$^{136}$Xe n-captures/muon')
plt.ylabel('Probability')
plt.semilogy()
plt.show()
fig.savefig(outdir / f"136Xe_multiplicity_distr.png", bbox_inches="tight", transparent=False, facecolor='white')